In [1]:
import numpy as np
import matplotlib.pyplot as plt
from os import path
#%matplotlib notebook

from brainpipe.system import study
from brainpipe.visual import BorderPlot, addLines
from brainpipe.feature import TF
from brainpipe.feature import PLV
from brainpipe import *

In [3]:
st = study('Olfacto')
pathdata = path.join(st.path, 'database/TS_E_all_by_odor_th40_art400_30_250_Good_Bad_RecScore/')
path2save = path.join(st.path, 'feature/8_PLV_RECSCORE_Good_Bad_across_time_subset/')

test = 'subset' 

if test == True:
    n_elec_list = {'VACJ' :[1,2]}
    subjects = ['VACJ']
    freq = [[2, 4], [5, 7]] 
    fname = ['delta', 'theta'] 
    
elif test == False :
    subjects = ['SEMC','PIRJ','LEFC','MICP','CHAF','VACJ'] 
    n_elec_list = {
    'CHAF' : 107,
    'VACJ' : 139, 
    'SEMC' : 107,
    'PIRJ' : 106,
    'LEFC' : 193,
    'MICP' : 105,
        }
    
elif test == 'subset': #HC, PHC, Amg, Pir
    subjects = ['SEMC','PIRJ','LEFC','MICP','VACJ'] 
    n_elec_list = {
    'VACJ' : [1,2,3,11,12,13,14,15,16,17,22,23,24,60,61,62],
    'SEMC' : [0,1,2,3,4],
    'PIRJ' : [0,1,2,3,4,11,12,13,14,15,16,22,23,24,25,26,33,34,35,36,37,38],
    'LEFC' : [0,1,11,12,13,14,22,23,24,25,26,27],
    'MICP' : [0,1,2,3,9,18,10,11,12,19,20,21,22,23,29,30,31,32,33,40,41,42,43,44],
        }
    
# Parameters for the PLV
freq = [[2, 4], [5, 7], [8, 13], [13, 30], [30, 60], [60, 150]] 
fname = ['delta', 'theta', 'alpha', 'beta', 'gamma30-60', 'gamma60-150'] 
n_perm = 1
sf = 512
method = 'hilbert'
conds = ['bad', 'good']

for su in subjects:
    for cond in conds:
        
        all_data, all_label = np.array([]), np.array([])
        # create data and labels matrices (to compute PLV on a subset of electrodes)
        for elec in n_elec_list[su]:
            data = np.load(path.join(pathdata, su+'_concat_odor_'+cond+'_bipo.npz'))['x'][elec]
            label = np.load(path.join(pathdata, su+'_concat_odor_'+cond+'_bipo.npz'))['label'][elec]
            data = data[np.newaxis,...]
            all_data = np.vstack((all_data,data)) if np.size(all_data) else data
            all_label = np.vstack((all_label,label)) if np.size(all_label) else label
        all_label = all_label.ravel()
    
        # compute the PLV for one cond, one freq
        for elec in n_elec_list[su]:
            for i,f in enumerate(freq):
                n_elec, n_pts, n_trials = all_data.shape[0], all_data.shape[1], all_data.shape[2]
                elec_data = np.load(path.join(pathdata, su+'_concat_odor_'+cond+'_bipo.npz'))['x'][elec]
                elec_data = elec_data[np.newaxis,...]
                elec_label = np.load(path.join(pathdata, su+'_concat_odor_'+cond+'_bipo.npz'))['label'][elec]

                defPlv = PLV(sf=sf, npts=n_pts, f=f, method=method, cycle=3)
                plv, pvalues = defPlv.get(all_data,elec_data,n_perm = n_perm)
                plv = np.squeeze(plv)
                np.save(path2save +su+'_'+cond+'_plv_'+fname[i]+'_elec_('+str(elec)+')_'+str(elec_label)+'.npy',plv)
                
                #Plot PLV values and labels
                plot_title = 'Plot PLV for '+su+' '+cond+' '+fname[i]+' elec('+str(elec)+') '+str(elec_label)
                fig = plt.figure(figsize=(9,6))
                fig.suptitle(plot_title, fontsize=16, y=1., fontweight='bold')
                step = 3700/ elec_data.shape[1]
                time = np.arange(-700, 3000, step)

                defPlv.plot2D(fig, plv, xvec=time, vmin=0, vmax=0.8,xlabel='Temps', 
                              ylabel='Electrodes', cmap='jet', cblabel='LTM - Olfactory')
                addLines(plt.gca(), vLines=[0], vColor=['black'], vWidth=[2], vShape=['-'])
                #print (all_label)
                plt.yticks(np.arange(len(all_label))+0.5,all_label)
                fig.savefig(path2save +su+'_'+cond+'_plv_'+fname[i]+'_elec_('+str(elec)+')_'+str(elec_label)+'.png',dpi=300, bbox_inches='tight')
                plt.clf()
                plt.close()
        del elec, all_data, all_label, elec_data, elec_label
                

-> Olfacto loaded
